In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk #machine learning 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #use string to create variables
        exec("%s = '%s'" % (filename[:-4],os.path.join(dirname, filename)))
        filename[:-4]
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predictpotentialspammersonfiverr/sample_submission.csv
/kaggle/input/predictpotentialspammersonfiverr/train.csv
/kaggle/input/predictpotentialspammersonfiverr/test.csv


In [2]:
train_data = pd.read_csv(train)

In [3]:
nan_cols = train_data.columns[train_data.isna().sum()>0].to_list()

In [4]:
train_data[nan_cols]=train_data[nan_cols].interpolate()

In [5]:
train_info = train_data.iloc[:,2:].describe()
no_deviation_cols = train_info.columns[train_info.loc['std'] == 0]
#train_info

In [6]:
train_data.drop(columns=no_deviation_cols,inplace=True)

In [ ]:
#train_data

In [ ]:
#train_data_cl1=train_data[train_data['label']==1]
#train_data1 = None#train_data

In [ ]:
#train_data1

In [ ]:
#cols = train_data_cl1.columns.to_list()
#cols = cols[2:]
#train_data[[train_data.iloc[:,2:] > train_data.iloc[:,2:].quantile(q=[0.10])]
#for i in cols:
#    lq = train_data_cl1[i].quantile(q=[0.011]).to_list()[0]
#    hq = train_data_cl1[i].quantile(q=[0.98]).to_list()[0]
#    train_data1 = train_data_cl1[(train_data_cl1[i] >= lq) & (train_data_cl1[i] <= hq)]
#    #print(hq , lq)
    


In [ ]:
#train_data_cl0=train_data[train_data['label']==0]
#train_data2 = None

In [ ]:
#cols = train_data_cl0.columns.to_list()
#cols = cols[2:]
#train_data[[train_data.iloc[:,2:] > train_data.iloc[:,2:].quantile(q=[0.10])]
#for i in cols:
#    lq = train_data_cl0[i].quantile(q=[0.011]).to_list()[0]
#    hq = train_data_cl0[i].quantile(q=[0.98]).to_list()[0]
#    train_data2 = train_data_cl0[(train_data_cl0[i] >= lq) & (train_data_cl0[i] <= hq)]
    #print(hq , lq)


In [ ]:
#train_data['user_id'][train_data['user_id'].duplicated() == True]

In [ ]:
#train_data2

In [14]:
train_data1 = train_data[train_data['label'] == 1]
train_data2 = train_data[train_data['label'] == 0]

In [8]:
#sample_frac = len(train_data1['user_id'])/len(train_data2['user_id'])

In [9]:
#train_data2 = train_data2.sample(frac=sample_frac)

In [15]:
len(train_data2['user_id'])

446477

In [16]:
train_data3=pd.concat([train_data1,train_data2],ignore_index=True,axis=0)

In [17]:
def normalize(df , method : "z-score" or "min-max" or "max"):
    cols = df.columns.to_list()
    cols=cols[2:]
    if method == 'z-score':
        for col in cols:
            df[col] = (df[col] - df[col].mean())/df[col].std()
    elif method == 'max':
        for col in cols:
            df[col] = df[col]/df[col].abs().max()
    elif method == 'min-max':
        for col in cols:
            df[col] = (df[col] - df[col].min())/(df[col].max() - df[col].min())
    return df

In [18]:
train_data3 = normalize(train_data3,"min-max")

In [ ]:
#train_data3

In [19]:
from sklearn.model_selection import train_test_split
y_train = train_data3.iloc[:,0]
X_train = train_data3.iloc[:,2:]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
#                                                    random_state=42)

In [20]:
no_cols=len(X_train.columns)

In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(loss="auto", learning_rate=0.1, 
                                       max_iter=200, max_leaf_nodes=no_cols, 
                                       max_depth=None, min_samples_leaf=20,
                                       l2_regularization=0.0, max_bins=255, 
                                       categorical_features=None, 
                                       monotonic_cst=None, warm_start=False, 
                                       early_stopping=False, scoring="loss", 
                                       validation_fraction=0.1, 
                                       n_iter_no_change=50, tol=1e-7, 
                                       verbose=0, random_state=42)
from sklearn.ensemble import RandomForestClassifier
model_r = RandomForestClassifier(n_estimators=100, 
                                 criterion="entropy", 
                                 max_depth=None, min_samples_split=2, 
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                 max_features="auto", max_leaf_nodes=None, 
                                 min_impurity_decrease=0.0, bootstrap=True, 
                                 oob_score=True, n_jobs=None, random_state=42, 
                                 verbose=0, warm_start=False, class_weight={0: 1, 1: 2}, 
                                 ccp_alpha=0.0, max_samples=None)
#from sklearn.neighbors import KNeighborsRegressor
#model = KNeighborsRegressor()

In [22]:
model.fit(X_train,y_train)

HistGradientBoostingClassifier(early_stopping=False, max_iter=200,
                               max_leaf_nodes=44, n_iter_no_change=50,
                               random_state=42)

In [ ]:
#model_r.fit(X_train,y_train)

In [ ]:
"""from sklearn import metrics

pred = model.predict(X_test)

print(metrics.classification_report(y_test , pred) )

print(metrics.mean_squared_error(y_test , pred ) )

print(metrics.mean_absolute_error(y_test , pred) )"""

In [ ]:
"""pred = model_r.predict(X_test)

print(metrics.classification_report(y_test , pred) )

print(metrics.mean_squared_error(y_test , pred ) )

print(metrics.mean_absolute_error(y_test , pred) )"""

In [ ]:
#trying to stack models
#from sklearn.compose import make_column_selector

#cat_selector = make_column_selector(dtype_include=object)
#num_selector = make_column_selector(dtype_include=np.number)


In [ ]:
#from sklearn.compose import make_column_transformer
#from sklearn.impute import SimpleImputer
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import OrdinalEncoder

#cat_tree_processor = OrdinalEncoder(
#    handle_unknown="use_encoded_value", unknown_value=-1
#)
#num_tree_processor = SimpleImputer(strategy="mean", add_indicator=True)

#tree_preprocessor = make_column_transformer(
#    (num_tree_processor, num_selector), (cat_tree_processor, cat_selector)
#)
#tree_preprocessor

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import StandardScaler

#cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
#num_linear_processor = make_pipeline(
#    StandardScaler(), SimpleImputer(strategy="mean", add_indicator=True)
#)

#linear_preprocessor = make_column_transformer(
#    (num_linear_processor, num_selector), (cat_linear_processor, cat_selector)
#)
#linear_preprocessor

In [ ]:
#from sklearn.linear_model import LassoCV
#
#lasso_pipeline = make_pipeline(linear_preprocessor, LassoCV())
#lasso_pipeline

In [ ]:
#from sklearn.ensemble import RandomForestRegressor

#rf_pipeline = make_pipeline(tree_preprocessor, RandomForestRegressor(random_state=42))
#rf_pipeline

In [ ]:
#from sklearn.ensemble import HistGradientBoostingRegressor

#gbdt_pipeline = make_pipeline(
#    tree_preprocessor, HistGradientBoostingRegressor(random_state=0)
#)
#gbdt_pipeline

In [ ]:
#from sklearn.ensemble import StackingRegressor
#from sklearn.linear_model import RidgeCV

#estimators = [
#    ("Random Forest", rf_pipeline),
#    ("Lasso", lasso_pipeline),
#    ("Gradient Boosting", gbdt_pipeline),
#]

#stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=RidgeCV())
#stacking_regressor

In [ ]:
'''import time
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict


def plot_regression_results(ax, y_true, y_pred, title, scores, elapsed_time):
    """Scatter plot of the predicted vs true targets."""
    ax.plot(
        [y_true.min(), y_true.max()], [y_true.min(), y_true.max()], "--r", linewidth=2
    )
    ax.scatter(y_true, y_pred, alpha=0.2)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines["left"].set_position(("outward", 10))
    ax.spines["bottom"].set_position(("outward", 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel("Measured")
    ax.set_ylabel("Predicted")
    extra = plt.Rectangle(
        (0, 0), 0, 0, fc="w", fill=False, edgecolor="none", linewidth=0
    )
    ax.legend([extra], [scores], loc="upper left")
    title = title + "\n Evaluation in {:.2f} seconds".format(elapsed_time)
    ax.set_title(title)


fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)

for ax, (name, est) in zip(
    axs, estimators + [("Stacking Regressor", stacking_regressor)]
):
    start_time = time.time()
    score = cross_validate(
        est, X, y, scoring=["r2", "neg_mean_absolute_error"], n_jobs=2, verbose=0
    )
    elapsed_time = time.time() - start_time

    y_pred = cross_val_predict(est, X, y, n_jobs=2, verbose=0)

    plot_regression_results(
        ax,
        y,
        y_pred,
        name,
        (r"$R^2={:.2f} \pm {:.2f}$" + "\n" + r"$MAE={:.2f} \pm {:.2f}$").format(
            np.mean(score["test_r2"]),
            np.std(score["test_r2"]),
            -np.mean(score["test_neg_mean_absolute_error"]),
            np.std(score["test_neg_mean_absolute_error"]),
        ),
        elapsed_time,
    )

plt.suptitle("Single predictors versus stacked predictors")
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()'''

In [23]:
test_data = pd.read_csv(test)

In [24]:
test_data.columns[test_data.isna().sum() >0]

Index([], dtype='object')

In [25]:
test_data.drop(columns=no_deviation_cols, inplace = True)

In [ ]:
#test_data

In [26]:
def normalize_test(df , method : "z-score" or "min-max" or "max"):
    cols = df.columns.to_list()
    cols=cols[1:]
    if method == 'z-score':
        for col in cols:
            df[col] = (df[col] - df[col].mean())/df[col].std()
    elif method == 'max':
        for col in cols:
            df[col] = df[col]/df[col].abs().max()
    elif method == 'min-max':
        for col in cols:
            df[col] = (df[col] - df[col].min())/(df[col].max() - df[col].min())
    return df

In [27]:
test_data1 = normalize_test(test_data,'min-max')

In [ ]:
#test_data1

In [28]:
Xt=test_data1.iloc[:,1:]

In [29]:
pred=model.predict(Xt)

In [30]:
df_pred=pd.DataFrame({'user_id':test_data.iloc[:,0],'prediction':pred})

In [31]:
import time

In [32]:
time1 = time.localtime()
filename = f'fiver_output{time1.tm_year}-{time1.tm_mon}-{time1.tm_mday}-{time1.tm_hour}-{time1.tm_min}.csv'

In [33]:
filename

'fiver_output2022-9-11-19-4.csv'

In [34]:
df_pred.to_csv(filename,index=False)

<a href="./fiver_output2022-9-11-19-4.csv"> Download </a>